In [31]:
from arcgis.gis import GIS
import datetime

"""accounts to keep from user/groups/items deletion"""
ignore_accounts_online = ['DavidJVitale', 'yjiang_geosaurus', 'amani_geosaurus', 'api_data_owner',
                   'bmajor_geosaurus', 'rsingh_geosaurus', 'rohitgeo', 'andrew887',
                   'cwhitmore_geosaurus', 'ArcGISPyAPIBot']

ignore_accounts_playground = ['andrew', 'andrew.chapkowski', 'dvitale', 'david.vitale',
                   'atma.mani', 'john.yaist', 'bill.major', 'YJiang',
                   'rohit.singh', 'rohitgeo', 'gbochenek_python',
                   'system_publisher', 'admin', 'portaladmin',
                   'Demo_User', 'First_User', 'Second_User',
                   'api_data_owner', 'arcgis_python', 'temp_execution']

"""accounts that you want to delete groups and items, but keep user"""
target_accounts_online = ['arcgis_python']
target_accounts_playground = ['arcgis_python']

"""data to publish"""
data_paths = ['\\archive\crdata\Geosaurus_datasets\data_prep\csv\Trailheads.csv']

"""create GIS connection via admin credentials"""
gis_online = GIS(profile="your_online_admin_profile")
gis_playground = GIS(profile='your_ent_admin_profile')

In [32]:
def delete_depending_items(dependent_item):
    """deletes the item's depending items, and then the item"""
    depending_items = None
    try:
        depending_items = dependent_item.dependent_to()
        if depending_items['list']:
            for item in depending_items['list']:
                delete_depending_items(item)
    except:
        print("=== could not get item list %s" % dependent_item.homepage)
        
    if dependent_item.protected:
        dependent_item.protect(False)
    try:
        print("=== deleting the item: %s" % dependent_item.homepage)
        dependent_item.delete()
    except:
        print("=== could not delete non-dependent item %s" % dependent_item.homepage)


def delete_items(user):
    """deletes the user items"""
    folders = [None] + user.folders
    for folder in folders:
        print("=== deleting inside folder: %s" % folder)
        for item in user.items(folder=folder, max_items=255):
            delete_depending_items(item)
    print("=== finished deleting items owned by " + user.username)


def delete_groups(gis, user):
    """deletes the user groups, and removes user from groups where user is a member of"""
    groups_for_deletion = gis.groups.get('query=owner:' + user.username)
    if groups_for_deletion is not None:
        for group in groups_for_deletion:
            try:
                print("=== deleting group %s" % group.groupid)
                group.delete()
            except:
                print("=== could not delete group %s" % group.groupid)

    for grp in user.groups:
        print("=== removing from group: %s" % grp.id)
        if grp.owner != user.username:
            try:
                grp.remove_users([user.username])
            except:
                print("=== User cannot be removed from group: %s" % grp.id)
    print("=== finished deleting groups owned by " + user.username)


def delete_for_users(gis, ignore_accounts, target_accounts):
    """deletes items and groups for users in target_accounts, and ignore others"""
    for user in gis.users.search():
        if user.username not in ignore_accounts and not user.username.startswith("esri_"):
            print("-*-*-*-*-*-*-Delete groups & items & user for %s -*-*-*-*-*-" % user.username)
            delete_items(user)
            delete_groups(gis, user)
            try:
                user.delete()
            except:
                print("could not delete user %s" % user.username)

        elif user.username in target_accounts:
            print("-*-*-*-*-*-*-Delete groups & items for %s -*-*-*-*-*-*-*-*-" % user.username)
            delete_items(user)
            delete_groups(gis, user)

        else:
            print("-*-*-*-*-*-*-*-*-No Delete for %s -*-*-*-*-*-*-*-*-*-*-" % user.username)

def publish_data(gis, ignore_accounts, target_accounts):
    """publish sample data"""
    return

In [33]:
delete_for_users(gis_online, ignore_accounts_online, target_accounts_online)
delete_for_users(gis_playground, ignore_accounts_playground, target_accounts_playground)

-*-*-*-*-*-*-*-*-No Delete for amani_geosaurus -*-*-*-*-*-*-*-*-*-*-
-*-*-*-*-*-*-*-*-No Delete for api_data_owner -*-*-*-*-*-*-*-*-*-*-
-*-*-*-*-*-*-*-*-No Delete for ArcGISPyAPIBot -*-*-*-*-*-*-*-*-*-*-
-*-*-*-*-*-*-Delete groups & items & user for arcgis_python -*-*-*-*-*-
=== deleting inside folder: None
=== deleting the item: https://geosaurus.maps.arcgis.com/home/item.html?id=53833b01e2be4184905af62631ab7496
=== deleting the item: https://geosaurus.maps.arcgis.com/home/item.html?id=60819d3897c946948de05cb38d9dc6b3
=== deleting the item: https://geosaurus.maps.arcgis.com/home/item.html?id=263baa26b6c348238a23cf464d4b0780
=== deleting the item: https://geosaurus.maps.arcgis.com/home/item.html?id=816080ac91454aefb44a0fcda79678fd
=== deleting the item: https://geosaurus.maps.arcgis.com/home/item.html?id=081630aaa53a4a0dae785f96680f46be
=== deleting the item: https://geosaurus.maps.arcgis.com/home/item.html?id=0e1d4abdba3a4b51be79a9cc93390f33
=== deleting the item: https://geosaurus.m

Group does not exist or is inaccessible.


=== removing from group: 00df55a8a6324c5aa4fc946d01aebb01
=== removing from group: 0199302a15064638a149a42e143dfcdd
=== removing from group: 0352e4178bb24751847ef9798abb6976
=== removing from group: 03587dc5f53b45669448b873d3fdcf43
=== removing from group: 10753adc0d4a4649b497b2fb28ad6483
=== removing from group: 13cde0d655a94e649a526dfed50ff31f
=== removing from group: 23972be2d52c4d4f8bf23f837f24c016
=== removing from group: 241e64de526a47609b5a9a19895b5655
=== removing from group: 30a6c48cfb4c47ca90de4a5c8a1a171d
=== removing from group: 33b9366851b54e978562eec8d6ea0b57
=== removing from group: 43c0ed0ccfd74866864f6f50b739fe0f
=== removing from group: 4cc6e91fc6fe48a890006168e8fed073
=== removing from group: 519b8fba0a4a40e09488725f4d815821
=== removing from group: 54f11bf7c49f4c60a0173ec9e5091751
=== removing from group: 5a6079c137cd40db995ce55d4d4e4506
=== removing from group: 5ed99a6c24ca43038f3cbcf667245099
=== removing from group: 698d7354d98d4e34981708e1cfb2e830
=== removing f

Unable to delete user. User must not own items or groups.


could not delete user arcgis_python
-*-*-*-*-*-*-*-*-No Delete for bmajor_geosaurus -*-*-*-*-*-*-*-*-*-*-
-*-*-*-*-*-*-*-*-No Delete for cwhitmore_geosaurus -*-*-*-*-*-*-*-*-*-*-
-*-*-*-*-*-*-*-*-No Delete for DavidJVitale -*-*-*-*-*-*-*-*-*-*-
-*-*-*-*-*-*-*-*-No Delete for rohitgeo -*-*-*-*-*-*-*-*-*-*-
-*-*-*-*-*-*-*-*-No Delete for rsingh_geosaurus -*-*-*-*-*-*-*-*-*-*-
-*-*-*-*-*-*-Delete groups & items & user for student1_uc2018ed -*-*-*-*-*-
=== deleting inside folder: None
=== finished deleting items owned by student1_uc2018ed


...

=== finished deleting groups owned by ArcGISPyAPIBot


Unable to delete user. Admins cannot delete themselves.


could not delete user ArcGISPyAPIBot
-*-*-*-*-*-*-Delete groups & items for arcgis_python -*-*-*-*-*-*-*-*-
=== deleting inside folder: None
=== deleting inside folder: {'username': 'arcgis_python', 'id': '267be085733e4ee0859699f8eceee4d9', 'title': 'Samples', 'created': 1533583177751}
=== finished deleting items owned by arcgis_python


Group does not exist or is inaccessible.


=== removing from group: 056407a2bacf48c5a8fc194a88041534
=== removing from group: 1981822cd41d498a9c7b5ea70db5c588
=== removing from group: e959fa22745f4511af136e13810af398
=== removing from group: 1228f2a5664449ecb800bfafceb6b7e7
=== removing from group: 0641018539bc45538cdf7fa43e7b36bc
=== removing from group: 65539cc4a1424913a743d2cafe5cedac
=== removing from group: cbecc140b7ea44d5bda562dc0653bff5
=== removing from group: dd0fc32aebf647c79a0cefa0400db768
=== removing from group: 622ab897d41d4dc79bbc202d22ff1234
=== removing from group: 99627d1766be412ebc888b0bbb55c6ee
=== removing from group: 4f7f44ba9ec44ecdbc5472be64bb56d0
=== removing from group: d94b6a9c7e4b433baf8ed41267caed06
=== removing from group: 8125f0d72fe8449d95aa76229a1390c2
=== removing from group: 3ba54c480102426aaf2b1e60f69e31bb
=== removing from group: 86e8dc2c2e7d43b6863a5c813f17780c
=== removing from group: 0d911dca04e948889cf8decd23655501
=== removing from group: 4a486c123d99471ab5a03e5824b62469
=== removing f

In [34]:
# csv_file = './data/LA_Hub_Datasets/Trailheads.csv'
csv_file = '/archive/crdata/Geosaurus_datasets/data_prep/csv/set2_Chicago.csv'
csv_item = gis_online.content.add({}, csv_file)

RuntimeError: File(/archive/crdata/Geosaurus_datasets/data_prep/csv/set2_Chicago.csv) not found.

In [16]:
csv_item

<Item title:"Trailheads" type:CSV owner:yjiang_geosaurus>

In [17]:
csv_lyr = csv_item.publish()

In [18]:
csv_lyr

<Item title:"Trailheads" type:Feature Layer Collection owner:yjiang_geosaurus>